In [ ]:
import streamlit as st
import pandas as pd
import requests
import geopandas as gpd
import folium

print(gpd.__version__)
exit()

# ===================================================================
# Función: load_data_from_api (con caching y limpieza inicial)
# ===================================================================
@st.cache_data
def load_data_from_api(limit: int = 50000) -> pd.DataFrame:
    """
    Carga datos desde la API de Socrata en formato JSON y los convierte en un DataFrame de pandas.
    Incluye normalización de columnas y conversión de tipos básicos.
    """
    api_url = f"https://www.datos.gov.co/resource/nudc-7mev.json?$limit={limit}"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)

        if df.empty:
            return df

        # Limpieza inicial de nombres de columnas
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(" ", "_")
            .str.replace(r"[()]", "", regex=True)
        )

        # Conversión de tipos básicos
        if "ano" in df.columns:
            df["ano"] = pd.to_numeric(df["ano"], errors="coerce")

        # Ejemplo de conversión para tasa de matriculados y cobertura neta
        for col in ["tasa_de_matriculados", "cobertura_neta"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")

        return df

    except requests.exceptions.RequestException as e:
        st.error(f"Error de conexión: {e}")
    except Exception as e:
        st.error(f"Error inesperado: {e}")
    return pd.DataFrame()

# ===================================================================
# Función: show_data_tab (con resumen exploratorio)
# ===================================================================
def show_data_tab():
    """
    Muestra la pestaña para cargar datos desde la API e incluye un pequeño resumen exploratorio.
    """
    st.header("📥 Carga de Datos del MEN vía API")

    st.markdown("""
    Este conjunto de datos proviene del portal 
    [datos.gov.co](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev).

    Presiona el botón para cargar los datos directamente desde la API.
    """)

    if st.button("🔄 Cargar datos"):
        with st.spinner("Cargando datos desde la API..."):
            df_raw = load_data_from_api()

        if not df_raw.empty:
            st.session_state['df_raw'] = df_raw

            # Resumen básico
            st.success(f"✅ Datos cargados: {df_raw.shape[0]} filas y {df_raw.shape[1]} columnas")
            if "ano" in df_raw.columns:
                st.write(f"📆 Años disponibles: {int(df_raw['ano'].min())} - {int(df_raw['ano'].max())}")
            if "entidad_territorial" in df_raw.columns:
                top_regiones = df_raw["entidad_territorial"].value_counts().head(5)
                st.write("🏆 **Top 5 regiones con más registros:**")
                st.dataframe(top_regiones)

            st.subheader("Vista preliminar de los datos")
            st.dataframe(df_raw.head(10))
        else:
            st.warning("No se encontraron datos o hubo un error en la carga.")
    else:
        st.info("Presiona el botón para iniciar la carga.")


ModuleNotFoundError: No module named 'streamlit'